In [55]:
import numpy as np
import math

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.threebody import flybys
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm
import poliastro.twobody.propagation as Propagation

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
T = 5*30*u.day;
a_at_saturn=((T**2*Saturn.k/(4*math.pi**2))**(1/3)).to(u.m)

In [12]:
float(a_at_saturn/(238000*u.km))


22.875575137363345

In [13]:
a_at_saturn.to

<Quantity 5444386882.692475 m>

In [4]:
(1*u.AU).to(u.m)

<Quantity 149597870700.0 m>

In [5]:
cassini_venus2 = Time("1999-06-24")
cassini_earth2 = Time("1999-08-18")
cassini_jupiter = Time("2000-12-30")
cassini_saturn = Time("2004-07-01")

#cassini transit
cassini_js_time = cassini_saturn-cassini_jupiter
cassini_ej_time = cassini_jupiter-cassini_earth2
cassini_ve_time = cassini_earth2-cassini_venus2

In [6]:
tmod=[0,0,0,0]
plot_on=0;

def trajectory_calculator(t=[0,0,0,0],plot_on=0,disp_on=0):
    #start with cassini style assumptions
    cassini_venus2 = Time("1999-06-24")
    cassini_earth2 = Time("1999-08-18")
    cassini_jupiter = Time("2000-12-30")
    cassini_saturn = Time("2004-07-01")

    #cassini transit
    cassini_js_time = cassini_saturn-cassini_jupiter
    cassini_ej_time = cassini_jupiter-cassini_earth2
    cassini_ve_time = cassini_earth2-cassini_venus2


    tmod_s  = t[0] #arrival time in saturn relative to 2020
    tmod_js = t[1] #transit times
    tmod_ej = t[2] 
    tmod_ve = t[3]


    time_js = tmod_js*u.year
    time_ej = tmod_ej*u.year
    time_ve = tmod_ve*u.year

    date_arrive_saturn = Time("2020-01-01", scale="utc")+tmod_s*u.year;


    #create target orbit
    o_sf = Orbit.from_body_ephem(Saturn, date_arrive_saturn)
    r_sf, v_sf = o_sf.rv()
    #o_sf.plot()
    if disp_on:
        print('computed saturn')

    ################## Jupiter ################## 

    #guess flyby date of Jupiter
    date_flyby_jupiter = date_arrive_saturn-time_js

    #construct j orbit
    o_j = Orbit.from_body_ephem(Jupiter, date_flyby_jupiter)
    #o_j.plot()

    #compute transfer lambert trajectory
    (v_jo, v_si), = izzo.lambert(Sun.k, o_j.r, o_sf.r, time_js)
    trx_js = Orbit.from_vectors(Sun, o_j.r, v_jo, epoch=date_flyby_jupiter)
    #trx_js.plot()
    if disp_on:
        print('computed J-S')
    ################## Earth ################## 
    #guess flyby date of Earth
    date_flyby_earth = date_arrive_saturn-time_js-time_ej

    #construct j orbit
    o_e2 = Orbit.from_body_ephem(Earth, date_flyby_earth)
    #o_e2.plot()

    #compute transfer lambert trajectory
    (v_eo, v_ji), = izzo.lambert(Sun.k, o_e2.r, o_j.r, time_ej)
    trx_ej = Orbit.from_vectors(Sun, o_e2.r, v_eo, epoch=date_flyby_earth)
    #trx_ej.plot()
    if disp_on:
        print('computed E-J')                            

    ################## Venus2 ################## 
    #guess flyby date of Venus2
    date_flyby_venus2 = date_arrive_saturn-time_js-time_ej-time_ve

    #construct j orbit
    o_v2 = Orbit.from_body_ephem(Venus, date_flyby_venus2)
    #o_v2.plot()

    #compute transfer lambert trajectory
    (v_v2o, v_ei), = izzo.lambert(Sun.k, o_v2.r, o_e2.r, time_ve)
    trx_v2e = Orbit.from_vectors(Sun, o_v2.r, v_v2o, epoch=date_flyby_venus2)
    #trx_v2e.plot()    
    if disp_on:
        print('computed V-E')                            

    ################## Sum delta v ##################                             
    delv_e = norm(v_eo-v_ei)
    delv_j = norm(v_jo-v_ji)
    delv_s = norm(v_si-v_sf)

    total_deltav=sum([delv_e,delv_j,delv_s]) 

    if disp_on:
        print('Total delta-v: ', total_deltav)

    ################## Plot ##################  

    if plot_on:
        op = OrbitPlotter2D()

        op.plot(o_v2,label="Venus2 Orbit")
        op.plot(o_e2,label="Earth2 Orbit")
        op.plot(o_j, label="Jupiter Orbit")
        op.plot(o_sf, label="Saturn Orbit")


        op.plot(trx_v2e, label="V2-E")
        op.plot(trx_ej, label="E-J")
        op.plot(trx_js, label="J-S")
        
    orbits = (o_v2,o_e2,o_j,o_sf)
    trajectories = (trx_v2e,trx_ej,trx_js)
    deltavs = (delv_e,delv_j,delv_s)
    times = (date_flyby_venus2,date_flyby_earth,date_flyby_jupiter,date_arrive_saturn)
    return (total_deltav,orbits,trajectories,deltavs,times)

def print_times(times):
    print("Venus:   ",times[0])
    print("Earth:   ",times[1])
    print("Jupiter: ",times[2])
    print("Saturn:  ",times[3])
    print("**")
    print("V-E: ", (times[1]-times[0]).to(u.year))
    print("E-J: ", (times[2]-times[1]).to(u.year))
    print("E-S: ", (times[3]-times[2]).to(u.year))
        



In [7]:
cassini_js_time.to(u.year).value

3.5017111567419574

In [8]:
t_guess_cassini = [27,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
sol=trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2046-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-01 18:01:09.184. Use Time(..., scale='tdb') instead.



computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  82.09792665510716 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-02-16 18:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-12-23 18:01:09.184. Use Time(..., scale='tdb') instead.



In [ ]:
sol=trajectory_calculator(tmod=[0,0,0,0],plot_on=0,disp_on=1)
if 1:
    op=OrbitPlotter2D()
    op.plot(sol[1][0],label='V')
    op.plot(sol[1][1],label='E')
    op.plot(sol[1][2],label='J')
    op.plot(sol[1][3],label='S')
    op.plot(sol[2][0],label='VE')
    op.plot(sol[2][1],label='EJ')
    op.plot(sol[2][2],label='JS')



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:01:09.186. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-27 00:01:09.182. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-05-15 00:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-03-21 00:01:09.186. Use Time(..., scale='tdb') instead.



computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  49.73689056630519 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



In [6]:
sol

(<Quantity 82.09792665510716 km / s>,
 (1 x 1 AU x 24.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-12-23 18:01:09.184 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2042-02-16 18:01:09.185 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2043-07-01 18:01:09.184 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2046-12-31 18:01:09.184 (TDB)),
 (30293036 x -2883088647 km x 24.0 deg (HCRS) orbit around Sun (☉) at epoch 2041-12-23 18:00:00.000 (UTC),
  1 x 10 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch 2042-02-16 18:00:00.000 (UTC),
  0 x 47 AU x 27.1 deg (HCRS) orbit around Sun (☉) at epoch 2043-07-01 18:00:00.000 (UTC)),
 (<Quantity 38.076032380696105 km / s>,
  <Quantity 29.27965312341461 km / s>,
  <Quantity 14.742241150996456 km / s>),
 (<Time object: scale='utc' format='iso' value=2041-12-23 18:00:00.000>,
  <Time object: scale='utc' format='iso' value=2042-02-16 18:00:00.000>,
  <Time object: scale='utc' format='iso' val

In [7]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [15]:
t_guess_cassini = [18,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
bounds = ((10,20),(0.01,5),(0.01,3),(0.01,1))
options = {'maxiter': 30,'disp': True}

def total_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[0])
    return sol[0].to(u.km/u.s).value

opt_sol = opt.minimize(total_deltav_calculator,x0=t_guess_cassini,options=options,bounds=bounds)

opt_sol

[ 18.           3.50171116   1.36892539   0.15058179]
79.18253603560625 km / s
[ 18.00000001   3.50171116   1.36892539   0.15058179]
79.18253455682253 km / s
[ 18.           3.50171117   1.36892539   0.15058179]
79.18253741852827 km / s
[ 18.           3.50171116   1.3689254    0.15058179]
79.18253723952036 km / s
[ 18.           3.50171116   1.36892539   0.1505818 ]
79.18253497196739 km / s
[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
5616.589019747007 km / s
[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
5616.588964607887 km / s
[  2.00000000e+01   1.00000100e-02   1.00000000e-02   1.00000000e+00]
5616.587345878905 km / s
[  2.00000000e+01   1.00000000e-02   1.00000100e-02   1.00000000e+00]
5616.585161113126 km / s

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2037-12-31 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-07-01 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2033-02-16 12:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-12-23 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 


[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000001e+00]
5616.589019167613 km / s
[ 18.74169365   2.20682116   0.86497223   0.46558584]
90.91188196574512 km / s
[ 18.74169366   2.20682116   0.86497223   0.46558584]
90.91188313406796 km / s
[ 18.74169365   2.20682117   0.86497223   0.46558584]
90.9118806279785 km / s
[ 18.74169365   2.20682116   0.86497224   0.46558584]
90.91188033992796 km / s
[ 18.74169365   2.20682116   0.86497223   0.46558585]
90.91188105110047 km / s
[ 18.33419339   2.91825776   1.14185345   0.29251677]
77.83777545685919 km / s
[ 18.3341934    2.91825776   1.14185345   0.29251677]
77.8377752079213 km / s
[ 18.33419339   2.91825777   1.14185345   0.29251677]
77.83777558198452 km / s
[ 18.33419339   2.91825776   1.14185346   0.29251677]
77.83777528508602 km / s
[ 18.33419339   2.91825776   1.14185345   0.29251678]
77.83777416774129 km / s
[ 18.47631719   2.70469692   1.17390917   0.73221881]


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2038-09-28 09:42:20.630. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-07-14 08:42:41.035. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2035-09-02 10:21:53.461. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2035-03-16 09:02:21.846. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

81.74550994042472 km / s
[ 18.4763172    2.70469692   1.17390917   0.73221881]
81.74551109843445 km / s
[ 18.47631719   2.70469693   1.17390917   0.73221881]
81.7455086454502 km / s
[ 18.47631719   2.70469692   1.17390918   0.73221881]
81.74550853962631 km / s
[ 18.47631719   2.70469692   1.17390917   0.73221882]
81.7455092174843 km / s
[ 18.37889723   2.851084     1.15193631   0.43082133]
76.47339745525795 km / s
[ 18.37889724   2.851084     1.15193631   0.43082133]
76.4733956713364 km / s
[ 18.37889723   2.85108401   1.15193631   0.43082133]
76.47339911186768 km / s
[ 18.37889723   2.851084     1.15193632   0.43082133]


...

27.068558324917362 km / s
[ 20.           3.4290545    1.90108617   0.16826689]
27.068558323721714 km / s
[ 20.           3.4290545    1.90108616   0.1682669 ]
27.068557606501543 km / s
[ 20.           3.45284209   1.87897398   0.1721752 ]
26.944396059658075 km / s
[ 20.00000001   3.45284209   1.87897398   0.1721752 ]
26.944395995007575 km / s
[ 20.           3.4528421    1.87897398   0.1721752 ]
26.944396050629628 km / s
[ 20.           3.45284209   1.87897399   0.1721752 ]
26.944396045432896 km / s
[ 20.           3.45284209   1.87897398   0.17217521]
26.94439596393055 km / s
[ 20.           3.40492324   1.92553238   0.17275405]
26.929084740338002 km / s
[ 20.00000001   3.40492324   1.92553238   0.17275405]
26.92908471506907 km / s
[ 20.           3.40492325   1.92553238   0.17275405]
26.929084690525244 km / s
[ 20.           3.40492324   1.92553239   0.17275405]
26.929084688425316 km / s
[ 20.           3.40492324   1.92553238   0.17275406]
26.929084755527878 km / s
[ 20.           

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-05 08:27:44.007. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-09-02 01:14:43.395. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-06-30 22:53:00.053. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-05 08:27:43.376. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

26.92837427492946 km / s
[ 20.           3.39882233   1.93161321   0.17276811]
26.92837427443158 km / s
[ 20.           3.39882233   1.9316132    0.17276812]
26.928374295555173 km / s
[ 20.           3.39830699   1.9321789    0.17276969]
26.928322089384356 km / s
[ 20.00000001   3.39830699   1.9321789    0.17276969]
26.92832201447039 km / s
[ 20.           3.398307     1.9321789    0.17276969]
26.92832208901571 km / s
[ 20.           3.39830699   1.93217891   0.17276969]
26.928322089014348 km / s
[ 20.           3.39830699   1.9321789    0.1727697 ]
26.928322089560293 km / s
[ 20.           3.39842047   1.93207053   0.17276996]
26.928322000845817 km / s
[ 20.00000001   3.39842047   1.93207053   0.17276996]
26.928321925575077 km / s
[ 20.           3.39842048   1.93207053   0.17276996]
26.92832200083983 km / s
[ 20.           3.39842047   1.93207054   0.17276996]
26.928322000840307 km / s
[ 20.           3.39842047   1.93207053   0.17276997]
26.92832200085186 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-07 17:27:54.953. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-09-02 00:56:06.130. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-06-30 22:26:00.760. Use Time(..., scale='tdb') instead.



      fun: 26.928322000845817
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.52707408e+00,  -5.98632255e-04,  -5.51025892e-04,
         6.04316597e-04])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 260
      nit: 30
   status: 1
  success: False
        x: array([ 20.        ,   3.39842047,   1.93207053,   0.17276996])

In [27]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [ ]:

options = {'maxiter': 30,'disp': False}
opt_sol = opt.minimize(total_deltav_calculator,x0=opt_sol.x,options=options)
opt_sol

In [16]:
opt_sol

      fun: 26.928322000845817
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.52707408e+00,  -5.98632255e-04,  -5.51025892e-04,
         6.04316597e-04])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 260
      nit: 30
   status: 1
  success: False
        x: array([ 20.        ,   3.39842047,   1.93207053,   0.17276996])

In [ ]:
sol=trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=0)

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2046-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-01 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-02-16 18:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-12-23 18:01:09.184. Use Time(..., scale='tdb') instead.




Total deltav:  82.09792665510716 km / s
Delta vs:  [38.076032380696105, 29.27965312341461, 14.742241150996456]

Times:
Venus:    2041-12-23 18:00:00.000
Earth:    2042-02-16 18:00:00.000
Jupiter:  2043-07-01 18:00:00.000
Saturn:   2046-12-31 18:00:00.000
**
V-E:  0.15058179329226556 yr
E-J:  1.3689253935660506 yr
E-S:  3.5017111567419574 yr



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



In [14]:
norm(sol[1][2].v).to(u.km/u.s)

<Quantity 13.433691807592876 km / s>

In [14]:
###############VERY GOOD SOLUTION############
tverygood = [27.238877,4.798,1.2879,0.1313]
sol=trajectory_calculator(t=tverygood,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  6.010478813049802 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:00:54.001. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-10 12:44:49.200. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-25 03:00:56.161. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 04:02:23.279. Use Time(..., scale='tdb') instead.



In [10]:
#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')


Total deltav:  6.010478813049802 km / s
Delta vs:  [0.19207023597361006, 0.0776495284180622, 5.74075904865813]

Times:
Venus:    2041-01-08 04:01:14.095
Earth:    2041-02-25 02:59:46.975
Jupiter:  2042-06-10 12:43:40.015
Saturn:   2047-03-28 23:59:44.815
**
V-E:  0.1313 yr
E-J:  1.2879 yr
E-S:  4.798 yr



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [41]:
trx_ve.p.to(u.AU)

<Quantity 1.3071194333827194 AU>

In [42]:
norm(trx_ve.r).to(u.AU)

<Quantity 0.7250798858414389 AU>

In [49]:
trx_ve.p.to(u.AU)

<Quantity 1.3071194333827194 AU>

In [44]:
trx_ve.ecc

<Quantity 0.8721853278753495>

In [45]:
trx_ve.inc.to(u.deg)

<Quantity 22.819458176477898 deg>

In [46]:
trx_ve.raan.to(u.deg)

<Quantity 0.6831220688742734 deg>

In [47]:
trx_ve.argp.to(u.deg)

<Quantity 87.26032791794375 deg>

In [48]:
trx_ve.nu.to(u.deg)

<Quantity -23.02119293489181 deg>

In [46]:
trx_ve.r

<Quantity [ 45776760.73223779, 90712263.57357913, 37955968.12664004] km>

In [51]:
norm(trx_ve.v-sol[1][0].v)

<Quantity 14.712681702425392 km / s>

In [54]:
calc_max_deltav(14.7*u.km/u.s,qratio=1.05, body=Venus)

<Quantity 5.624830542554564 km / s>

In [90]:
def calcerror(x):
    
    vexp = trx_ve.v
    vin=[x[0],x[1],x[2]]*u.km/u.s
    (vout,beta) = flybys.compute_flyby(v_spacecraft=vin-sol[1][0].v,v_body=sol[1][0].v,k=Venus.k,r_p=1.1*Venus.R)

    return norm(vout-vexp).to(u.km/u.s).value

In [91]:
vin_to_venus_sol=opt.minimize(calcerror,-trx_ve.v.value)

In [92]:
vin_to_venus_sol

      fun: 1.4483998570554667e-08
 hess_inv: array([[  7.90310219e-08,   3.71193252e-08,  -1.13515533e-08],
       [  3.71193252e-08,   3.37181311e-08,  -4.62236648e-09],
       [ -1.13515533e-08,  -4.62236648e-09,   3.40671562e-08]])
      jac: array([ 0.10780507,  0.03937952,  0.42328965])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 801
      nit: 37
     njev: 158
   status: 2
  success: False
        x: array([-77.7641715 ,  28.74623527,  12.23952858])

In [93]:
vin_to_venus=(vin_to_venus_sol.x)*u.km/u.s

In [94]:
trx_ve.v

<Quantity [-46.29153109, 10.88620178,  4.81238474] km / s>

In [95]:
vin_to_venus

<Quantity [-77.7641715 , 28.74623527, 12.23952858] km / s>

In [88]:
norm(trx_ve.v-vin_to_venus)

<Quantity 36.94144495584995 km / s>

In [96]:
(vout,beta) = flybys.compute_flyby(v_spacecraft=vin_to_venus-sol[1][0].v,v_body=sol[1][0].v,k=Venus.k,r_p=1.1*Venus.R)

In [97]:
vout

<Quantity [-46.2915311 , 10.88620177,  4.81238474] km / s>

In [106]:
beta

<Quantity 0.37004250279957 rad>

In [105]:
norm(calc_max_deltav(body=Venus,qratio=0.2,v_inf=vin_to_venus-sol[1][0].v))

<Quantity 21.19419960826762 km / s>

In [ ]:
trx_ve_half = trx_ve.propagate(t_ve/2)

In [ ]:
trx_ve_half.plot()

In [ ]:
norm(o_e.v.to(u.km/u.s))

In [ ]:
print_times(sol[4])

In [48]:
sol

(<Quantity 6.010478813049802 km / s>,
 (1 x 1 AU x 24.5 deg (ICRS) orbit around Sun (☉) at epoch 2041-01-08 04:02 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-02-25 03:00 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2042-06-10 12:44 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2047-03-29 00:00 (TDB)),
 (1 x 10 AU x 22.8 deg (HCRS) orbit around Sun (☉) at epoch 2041-01-08 04:01:14.095 (UTC),
  1 x 10 AU x 22.6 deg (HCRS) orbit around Sun (☉) at epoch 2041-02-25 02:59:46.975 (UTC),
  1 x 10 AU x 22.4 deg (HCRS) orbit around Sun (☉) at epoch 2042-06-10 12:43:40.015 (UTC)),
 (<Quantity 0.19207023597361006 km / s>,
  <Quantity 0.0776495284180622 km / s>,
  <Quantity 5.74075904865813 km / s>),
 (<Time object: scale='utc' format='iso' value=2041-01-08 04:01:14.095>,
  <Time object: scale='utc' format='iso' value=2041-02-25 02:59:46.975>,
  <Time object: scale='utc' format='iso' value=2042-06-10 12:43:40.015>,
  <Time object:

In [34]:
(Time("2047-03-29", scale="utc")-Time("2020-01-01", scale="utc")).to(u.year)

<Quantity 27.238877481177276 yr>

In [52]:
(norm(sol[1][3].v).to(u.km/u.s))**2

<Quantity 83.6607810811809 km2 / s2>

In [27]:
norm(sol[1][1].v).to(u.km/u.s)

<Quantity 29.940786635191444 km / s>

In [85]:
Earth.R

<<class 'astropy.constants.constant.Constant'> name='Earth equatorial radius' value=6378136.6 uncertainty=0.0001 unit='m' reference='IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009'>

In [90]:
earthflybyConstraint(sol)

35.717495055476995 km / s
29.94078663519144 km / s
[ -0.40886024  36.70086965  16.14330927] km / s
40.09647658080282 km / s
v_inf
21.02773372611768 km / s
max deltav
0.5861195770886941 km / s


60.938921768982425

In [81]:
v_inf = 3.5*u.km/u.s;
qmin = 10*6000*u.km;
2*v_inf*(1+((v_inf)**2*qmin/(Earth.k)).to(u.m/u.m))**-1

<Quantity 2.4613638013139316 km / s>

In [88]:
calc_max_deltav(3.5*u.km/u.s)

<Quantity 2.3647348537123474 km / s>

In [70]:
import math
math.sin(20*math.pi/180)

0.3420201433256687

In [15]:
tverygood

[27.238877, 4.798, 1.2879, 0.1313]

In [39]:
t_guess_cassini = [-15.5,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]

t_guess_online_traj_tool = [20, 7.93, 2.19,0.15]
t_guess_online_traj_tool = [12, 2, 2,0.15]


#t_guess = opt_sol.x

t_guess=tverygood

#t_guess=t_guess_online_traj_tool

bounds = ((27,32),(4,10),(0.6,1.5),(0.01,1))

options = {'maxiter': 50,'disp': True}

def calc_max_deltav(v_inf,qratio=10, body=Earth):
    return 2*v_inf*(1+((v_inf)**2*(qratio*body.R)/(body.k)).to(u.m/u.m))**-1

def saturn_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[3])
    
    
    
    return sol[3][2].to(u.km/u.s).value


def weighted_detlav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[3])
    
    
    
    return 10*sol[3][2].to(u.km/u.s).value + sum(sol[3]).to(u.km/u.s).value



def earthflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavearth = sol[3][0].to(u.km/u.s)
    print('deltav about earth')
    print(deltavearth)
    v_earth = sol[1][1].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_earth = sol[2][1].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_earth-v_earth).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Earth)
    print(max_deltav)
    
    return float((max_deltav-deltavearth)/max_deltav)


def jupiterflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavjup = sol[3][1].to(u.km/u.s)
    print('deltav about jupiter')
    print(deltavjup)
    v_jup = sol[1][2].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_jup = sol[2][2].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_jup-v_jup).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Jupiter)
    print(max_deltav)
    
    return float((max_deltav-deltavjup)/max_deltav)


earth_constraint ={'type':'ineq','fun':earthflybyConstraint}
jupiter_constraint ={'type':'ineq','fun':jupiterflybyConstraint}

# with constraints
#opt_sol = opt.minimize(saturn_deltav_calculator,x0=t_guess,options=options,bounds=bounds,constraints=(earth_constraint,jupiter_constraint))


#without constraints
opt_sol = opt.minimize(weighted_detlav_calculator,x0=t_guess,options=options,bounds=bounds)
opt_sol

[ 27.238877   4.798      1.2879     0.1313  ]
(<Quantity 0.19207023597361006 km / s>, <Quantity 0.0776495284180622 km / s>, <Quantity 5.74075904865813 km / s>)
[ 27.23887701   4.798        1.2879       0.1313    ]
(<Quantity 0.1920719809795592 km / s>, <Quantity 0.07764965131410509 km / s>, <Quantity 5.740759067763275 km / s>)
[ 27.238877     4.79800001   1.2879       0.1313    ]
(<Quantity 0.1920684909782443 km / s>, <Quantity 0.07764941284787588 km / s>, <Quantity 5.740759021965418 km / s>)
[ 27.238877     4.798        1.28790001   0.1313    ]
(<Quantity 0.19206846934860486 km / s>, <Quantity 0.07764942857254893 km / s>, <Quantity 5.74075904865813 km / s>)
[ 27.238877     4.798        1.2879       0.13130001]
(<Quantity 0.19207078122130178 km / s>, <Quantity 0.0776495284180622 km / s>, <Quantity 5.74075904865813 km / s>)
[  2.70000000e+01   1.00000000e+01   1.50000000e+00   1.00000000e-02]
(<Quantity 823.6751332445243 km / s>, <Quantity 13.019320812165414 km / s>, <Quantity 2.7327091

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:00:54.001. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-10 12:44:49.200. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-25 03:00:56.161. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 04:02:23.279. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

[  2.70000000e+01   1.00000000e+01   1.50000001e+00   1.00000000e-02]
(<Quantity 823.6751343140728 km / s>, <Quantity 13.019320782397914 km / s>, <Quantity 2.7327091918704483 km / s>)
[  2.70000000e+01   1.00000000e+01   1.50000000e+00   1.00000100e-02]
(<Quantity 823.6743080020333 km / s>, <Quantity 13.019320812165414 km / s>, <Quantity 2.7327091918704483 km / s>)
[ 27.13447573   7.07153579   1.38059837   0.0782858 ]
(<Quantity 109.30037614345737 km / s>, <Quantity 10.312594147763326 km / s>, <Quantity 2.071028544744479 km / s>)
[ 27.13447574   7.07153579   1.38059837   0.0782858 ]
(<Quantity 109.30037545156526 km / s>, <Quantity 10.312594125033874 km / s>, <Quantity 2.071028550146239 km / s>)
[ 27.13447573   7.0715358    1.38059837   0.0782858 ]
(<Quantity 109.30037683534947 km / s>, <Quantity 10.312594173827955 km / s>, <Quantity 2.071028537879795 km / s>)
[ 27.13447573   7.07153579   1.38059838   0.0782858 ]
(<Quantity 109.30037672738291 km / s>, <Quantity 10.312594004588812 km / s

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-02-18 20:50:00.475. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-01-23 23:45:02.774. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2038-09-06 17:25:31.533. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2038-08-09 03:10:19.707. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

(<Quantity 95.86788634265395 km / s>, <Quantity 8.304740251034538 km / s>, <Quantity 4.090502402590551 km / s>)
[ 27.20645342   5.50408489   1.31668905   0.11483554]
(<Quantity 95.86788629787536 km / s>, <Quantity 8.304740179192056 km / s>, <Quantity 4.090502421467264 km / s>)
[ 27.20645342   5.50408489   1.31668904   0.11483555]
(<Quantity 95.86787946458304 km / s>, <Quantity 8.304740050446918 km / s>, <Quantity 4.090502421467264 km / s>)
[ 27.23348227   4.91548059   1.29269001   0.12856059]
(<Quantity 29.091644855980707 km / s>, <Quantity 2.512093375938081 km / s>, <Quantity 5.425792887055679 km / s>)
[ 27.23348228   4.91548059   1.29269001   0.12856059]
(<Quantity 29.09164267751426 km / s>, <Quantity 2.512093206957069 km / s>, <Quantity 5.425792905456216 km / s>)
[ 27.23348227   4.9154806    1.29269001   0.12856059]
(<Quantity 29.091647034446943 km / s>, <Quantity 2.512093547601722 km / s>, <Quantity 5.4257928618364994 km / s>)
[ 27.23348227   4.91548059   1.29269002   0.12856059]
(

...

(<Quantity 0.16839702350284824 km / s>, <Quantity 5.351054106907404 km / s>, <Quantity 4.348262206702651 km / s>)
[ 31.95694041   9.54057545   1.26427863   0.13083898]
(<Quantity 0.16839641208726924 km / s>, <Quantity 5.351054120042175 km / s>, <Quantity 4.348262219276433 km / s>)
[ 31.9569404    9.54057546   1.26427863   0.13083898]
(<Quantity 0.16839763494088092 km / s>, <Quantity 5.35105410219443 km / s>, <Quantity 4.3482621931096075 km / s>)
[ 31.9569404    9.54057545   1.26427864   0.13083898]
(<Quantity 0.1683976119239376 km / s>, <Quantity 5.351053984269307 km / s>, <Quantity 4.348262206702651 km / s>)
[ 31.9569404    9.54057545   1.26427863   0.13083899]
(<Quantity 0.16839718768430512 km / s>, <Quantity 5.351054106907404 km / s>, <Quantity 4.348262206702651 km / s>)
[ 31.95689454   9.54039419   1.26426752   0.13084257]
(<Quantity 0.16278401457381006 km / s>, <Quantity 5.351217681820393 km / s>, <Quantity 4.348450946486594 km / s>)
[ 31.95689455   9.54039419   1.26426752   0.130

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-01 13:52:27.478. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-24 19:12:28.324. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 00:16:24.049. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2051-12-16 06:09:24.452. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

(<Quantity 30.91083176263207 km / s>, <Quantity 6.468334956744172 km / s>, <Quantity 4.179645464739438 km / s>)
[ 32.           9.70728912   1.2741973    0.12744448]
(<Quantity 30.910831691359412 km / s>, <Quantity 6.468334772766769 km / s>, <Quantity 4.179645477764635 km / s>)
[ 32.           9.70728912   1.27419729   0.12744449]
(<Quantity 30.910827632774154 km / s>, <Quantity 6.468334812890616 km / s>, <Quantity 4.179645477764635 km / s>)
[ 31.95689648   9.54040167   1.26426796   0.13084242]
(<Quantity 0.16288128336336466 km / s>, <Quantity 5.351211061864574 km / s>, <Quantity 4.348443207863794 km / s>)
[ 31.95689649   9.54040167   1.26426796   0.13084242]
(<Quantity 0.16288110139027456 km / s>, <Quantity 5.35121107529161 km / s>, <Quantity 4.348443220438009 km / s>)
[ 31.95689648   9.54040168   1.26426796   0.13084242]
(<Quantity 0.16288146536799808 km / s>, <Quantity 5.351211056859168 km / s>, <Quantity 4.34844319426999 km / s>)
[ 31.95689648   9.54040167   1.26426797   0.13084242

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-01 15:03:37.196. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-24 20:29:28.229. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 01:31:30.704. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2051-12-16 06:09:26.051. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

      fun: 53.3469621048787
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.15171588,   0.87548528,   1.30651188,  32.19906191])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 245
      nit: 30
   status: 0
  success: True
        x: array([ 31.95689458,   9.54039434,   1.26426753,   0.13084257])

In [181]:
#sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)
sol = trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  13.004790382447544 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2004-07-01 15:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2000-12-30 15:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 1999-08-18 15:01:09.183. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 1999-06-24 15:01:09.184. Use Time(..., scale='tdb') instead.



In [182]:
opt_sol.x

array([-12.09371764,   6.54631927,   1.71418377,   0.11473997])

In [183]:
calc_max_deltav(9*u.km/u.s)

<Quantity 1.2892988989720653 km / s>

In [40]:
sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)

print(opt_sol)

#sol = trajectory_calculator([ 21.82450945,   9.33104326,   2.39418937,   0.2])

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_js/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  9.862454269744987 km / s
      fun: 53.3469621048787
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.15171588,   0.87548528,   1.30651188,  32.19906191])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 245
      nit: 30
   status: 0
  success: True
        x: array([ 31.95689458,   9.54039434,   1.26426753,   0.13084257])

Total deltav:  9.862454269744987 km / s
Delta vs:  [0.16278594390086382, 5.351217542330753, 4.348450783513371]

Times:
Venus:    2041-01-08 01:30:21.520
Earth:    2041-02-24 20:28:19.044
Jupiter:  2042-06-01 15:02:28.011
Saturn:   2051-12-16 06:08:16.552
**
V-E:  0.13084257118186363 yr
E-J:  1.264267528819149 yr
E-S:  9.540394343692068 yr



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2051-12-16 06:09:25.735. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-01 15:03:37.196. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-24 20:29:28.229. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 01:31:30.704. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coord

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [197]:
trx_js_final = trx_js.propagate(t_js)

In [198]:
trx_js_final.v

<Quantity [-1.51840329,-5.6412066 ,-2.37581518] km / s>

In [199]:
o_s=sol[1][3]

In [200]:
o_s.v.to(u.km/u.s)

<Quantity [-5.06757586,-7.97630756,-3.07637505] km / s>

In [209]:
norm(trx_js_final.v-o_s.v)

<Quantity 4.305822410235878 km / s>

In [215]:
(Saturn.k/(4*Saturn.R))**0.5

<Quantity 12543.679524544172 m / s>

In [185]:
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()



Total deltav:  13.004790382447544 km / s
Delta vs:  [5.637569395491224, 2.0019564648133197, 5.365264522143001]



In [34]:
earthflybyConstraint(opt_sol.x)
print('jup')
jupiterflybyConstraint(opt_sol.x)

deltav about earth
0.36505968741801403 km / s
max deltav
2.6741868248894405 km / s
jup
deltav about jupiter
7.337152345333837 km / s
max deltav
16.370992735511038 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2052-01-01 00:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-04-16 08:41:00.083. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-25 07:11:26.672. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-11 00:05:39.699. Use Time(..., scale='tdb') instead.



0.5518199498422293